### 반복을 위한 Sacred
 - 목적  
    - 머신러닝 모델링에서 사용되는 config, parameter등을 손쉽게 저장할 수 있도록 도와주는 도구  

 - 이유  
    - 다양한 실험을 빠르게 진행하며, 손으로 기록하지 않고 자동으로 기록될 수 있도록 도와주는 도구  
    
 - 특징  
    - 환경설정을 다시 reproduce할 수 있도록 도와주는 도구
        - 실험의 모든 파라미터 추적
        - 여러 설정에 대해 쉽게 실험을 할 수 있음
        - DB에 각 실행의 설정을 저장
        - 결과를 reproduce  
        
    - 옴니보드등을 통해 저장된 값을 시각화할 수 있음
        - 옴니보드 : sacred에서 저장된 파리미터를 테이블로 보여주고 성능을 표시

#### sacred의 main mechanisms
- ConfigScopes : 함수의 local변수를 편리하게 다룰 수 있음. @ex.config 데코레이터로 사용
- Config Injection : 모든 함수에 있는 설정을 접근할 수 있음
- Command-line interface : 커맨드 라인으로 파라미터를 바꿔서 실행할 수 있음
- Observers : 실험의 모든 정보를 Observers에게 제공해 저장
- Automatic seeding : 실험의 무작위를 컨트롤할 때 도와줌

### sacred 공식 홈페이지 예시

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
from ipywidgets import interact
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import os

from numpy.random import permutation
from sklearn import svm, datasets
from sacred import Experiment
from sacred.observers import FileStorageObserver

plt.style.use('ggplot')
warnings.filterwarnings('ignore')

PROJECT_ID='new-york-taxi-309501' # 여기에 여러분들의 프로젝트 ID를 넣어주세요

In [2]:
ex = Experiment('iris_rbf_svm', interactive=True)

@ex.config
def cfg():
    C = 1.0
    gamma = 0.7

@ex.main
def run(C, gamma):
    iris = datasets.load_iris()
    per = permutation(iris.target.size)
    iris.data = iris.data[per]
    iris.target = iris.target[per]
    clf = svm.SVC(C, 'rbf', gamma=gamma)
    clf.fit(iris.data[:90],
          iris.target[:90])
    return clf.score(iris.data[90:],
                   iris.target[90:])

In [3]:
run_result = ex.run()

WARNING - iris_rbf_svm - No observers have been added to this run
INFO - iris_rbf_svm - Running command 'run'
INFO - iris_rbf_svm - Started
INFO - iris_rbf_svm - Result: 0.9666666666666667
INFO - iris_rbf_svm - Completed after 0:00:00


In [4]:
run_result.config

{'C': 1.0, 'gamma': 0.7, 'seed': 629063574}

In [5]:
run_result.result

0.9666666666666667

In [6]:
run_result.experiment_info

{'name': 'iris_rbf_svm',
 'base_dir': 'C:\\Users\\SANGMIN\\git\\뉴욕 택시 수요 예측',
 'sources': [],
 'dependencies': ['ipython==7.8.0',
  'ipywidgets==7.5.1',
  'matplotlib==3.3.3',
  'numpy==1.16.5',
  'pandas==0.25.1',
  'sacred==0.7.4',
  'scikit-learn==0.21.3',
  'seaborn==0.9.0'],
 'repositories': [],
 'mainfile': None}

### 기존 데이터와 통합

In [7]:
ex = Experiment('nyc-demand-prediction', interactive=True)

experiment_dir = os.path.join('./', 'experiments')
if not os.path.isdir(experiment_dir): 
    os.makedirs(experiment_dir)
ex.observers.append(FileStorageObserver.create(experiment_dir))

### 데이터 전처리

In [8]:
base_query = """
WITH base_data AS 
(
  SELECT nyc_taxi.*, gis.* EXCEPT (zip_code_geom)
  FROM (
    SELECT *
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
    WHERE 
        EXTRACT(MONTH from pickup_datetime) = 1
        and pickup_latitude  <= 90 and pickup_latitude >= -90
    ) AS nyc_taxi
  JOIN (
    SELECT zip_code, state_code, state_name, city, county, zip_code_geom
    FROM `bigquery-public-data.geo_us_boundaries.zip_codes`
    WHERE state_code='NY'
    ) AS gis 
  ON ST_CONTAINS(zip_code_geom, st_geogpoint(pickup_longitude, pickup_latitude))
)

SELECT 
    zip_code,
    DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour,
    EXTRACT(MONTH FROM pickup_datetime) AS month,
    EXTRACT(DAY FROM pickup_datetime) AS day,
    CAST(format_datetime('%u', pickup_datetime) AS INT64) -1 AS weekday,
    EXTRACT(HOUR FROM pickup_datetime) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend,
    COUNT(*) AS cnt
FROM base_data 
GROUP BY zip_code, pickup_hour, month, day, weekday, hour, is_weekend
ORDER BY pickup_hour
"""

base_df = pd.read_gbq(query=base_query, dialect='standard', project_id=PROJECT_ID)

WARNING - google.auth.compute_engine._metadata - Compute Engine Metadata server unavailable onattempt 1 of 3. Reason: timed out
WARNING - google.auth.compute_engine._metadata - Compute Engine Metadata server unavailable onattempt 2 of 3. Reason: timed out
WARNING - google.auth.compute_engine._metadata - Compute Engine Metadata server unavailable onattempt 3 of 3. Reason: timed out
WARNING - google.auth._default - Authentication failed using Compute Engine authentication due to unavailable metadata server.
Downloading: 100%|██████████████████████████████████████████████████████████| 87020/87020 [00:05<00:00, 15757.00rows/s]
INFO - pandas_gbq.gbq - Total time taken 7.77 s.
Finished at 2021-04-19 12:50:59.


In [9]:
base_df

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt
0,10004,2015-01-01 00:00:00,1,1,3,0,0,139
1,10039,2015-01-01 00:00:00,1,1,3,0,0,9
2,10174,2015-01-01 00:00:00,1,1,3,0,0,16
3,10037,2015-01-01 00:00:00,1,1,3,0,0,26
4,11221,2015-01-01 00:00:00,1,1,3,0,0,34
...,...,...,...,...,...,...,...,...
87015,10018,2015-01-31 23:00:00,1,31,5,23,1,566
87016,10022,2015-01-31 23:00:00,1,31,5,23,1,1095
87017,10021,2015-01-31 23:00:00,1,31,5,23,1,580
87018,10170,2015-01-31 23:00:00,1,31,5,23,1,33


### feature engineering

In [10]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(base_df[['zip_code']])
ohe_output = enc.transform(base_df[['zip_code']]).toarray()
ohe_df = pd.concat([base_df, pd.DataFrame(ohe_output, columns='zip_code_'+enc.categories_[0])], axis=1)
ohe_df['log_cnt'] = np.log10(ohe_df['cnt'])

### train/ test 데이터 나누기

In [11]:
def split_train_test(df, date):
    train_df = df[df['pickup_hour'] < date]
    test_df = df[df['pickup_hour'] >= date]
    return train_df, test_df

In [12]:
train_df, test_df = split_train_test(ohe_df,'2015-01-24')

In [13]:
train_df.head()

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,...,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801,log_cnt
0,10004,2015-01-01,1,1,3,0,0,139,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.143015
1,10039,2015-01-01,1,1,3,0,0,9,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.954243
2,10174,2015-01-01,1,1,3,0,0,16,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.204120
3,10037,2015-01-01,1,1,3,0,0,26,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.414973
4,11221,2015-01-01,1,1,3,0,0,34,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.531479


In [14]:
test_df.head()

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,...,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801,log_cnt
65118,10162,2015-01-24,1,24,5,0,1,5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.698970
65119,10038,2015-01-24,1,24,5,0,1,145,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.161368
65120,10025,2015-01-24,1,24,5,0,1,332,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.521138
65121,11217,2015-01-24,1,24,5,0,1,91,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.959041
65122,10451,2015-01-24,1,24,5,0,1,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.477121


In [15]:
del train_df['zip_code']
del train_df['pickup_hour']
del test_df['zip_code']
del test_df['pickup_hour']

In [16]:
y_train_raw = train_df.pop('cnt')
y_train_log = train_df.pop('log_cnt')
y_test_raw = test_df.pop('cnt')
y_test_log = test_df.pop('log_cnt')

In [17]:
y_true = y_test_raw.values.copy()
x_train = train_df.copy()
x_test = test_df.copy()

In [18]:
def evaluation(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    score = pd.DataFrame([mape, mae, mse], index=['mape', 'mae', 'mse'], columns=['score']).T
    return score

### 실험 설정

In [19]:
@ex.config
def config():
    fit_intercept=True
    normalize=False

In [20]:
@ex.capture
def get_model(fit_intercept, normalize):
    return LinearRegression(fit_intercept, normalize)

In [21]:
@ex.main
def run(_log, _run):
    lr_reg = get_model()
    lr_reg.fit(x_train, y_train_raw)
    pred = lr_reg.predict(x_test)

    _log.info("Predict End")
    score = evaluation(y_test_raw, pred)
    _run.log_scalar('model_name', lr_reg.__class__.__name__)
    
    _run.log_scalar('metrics', score)
    
    return score.to_dict()


In [22]:
experiment_result = ex.run()

INFO - nyc-demand-prediction - Running command 'run'
INFO - nyc-demand-prediction - Started run with ID "4"
INFO - run - Predict End
INFO - nyc-demand-prediction - Result: {'mape': {'score': 116835786436.73732}, 'mae': {'score': 1194321456.3110764}, 'mse': {'score': 1.3583082466004655e+21}}
INFO - nyc-demand-prediction - Completed after 0:00:01


In [23]:
experiment_result.config

{'fit_intercept': True, 'normalize': False, 'seed': 255598109}

### experiment 확인하기 위한 parser

- Experiment에서 log찍는 방식에 따라 사용할 함수가 다름  

1) _run.log_scalar에 metrics을 저장하는 경우  

In [24]:
def parsing_output(ex_id):
    with open(f'./experiments/{ex_id}/metrics.json') as json_file:
        json_data = json.load(json_file)
    with open(f'./experiments/{ex_id}/config.json') as config_file:
        config_data = json.load(config_file)
    
    output_df = pd.DataFrame(json_data['model_name']['values'], columns=['model_name'], index=['score'])
    output_df['experiment_num'] = ex_id
    output_df['config'] = str(config_data)
    metric_df = pd.DataFrame(json_data['metrics']['values'][0]['values'])
    
    output_df = pd.concat([output_df, metric_df], axis=1)
    output_df = output_df.round(2)
    return output_df

2) @ex.main의 함수에 결과를 return하는 경우

In [25]:
def parsing_output(ex_id):
    with open(f'./experiments/{ex_id}/run.json') as json_file:
        json_data = json.load(json_file)
    output = pd.DataFrame(json_data['result'])
    return output

In [29]:
parsing_output(4)

,mae,mape,mse
score,1.194321e+09,1.168358e+11,1.358308e+21
